# 2.1. Кортежи и списки (tuple, list)

List - динамический массив, кразмер которого может меняться. Благодаря объектной модели Питона может содержать объекты произвольной природы.

Вследствие динамической природа данного контейнера, временную сложность операций обычно оценивают по Amortized worst case.

In [1]:
a = list()

a.append(1)
# O(1)

a.extend([2])
# O(k)

a[0]
# O(1)

a[0] = 5
# O(1)

a.insert(2, 99)
# O(k)

a.pop(1)
# O(n)

a.index(5)
# O(n)

a.count(1)
# O(n)

a.sort(key=None, reverse=True)
# O(n log n)


In [2]:
a

[99, 5]

Чтобы сохранить время в дальнейшем, список выделяет дополнительное место по мере того, как мы добавляем новые элементы. Число элементов, которые выделяются для списка по мере добавления новых данных описывается как:

M=(N>>3)+(N<9?3:6)

Из этого следует, что, по возможности, следует помнить об overhead'ах создания списков и списков списков.

Кортежи же, напротив, статические неизменяемые контейнеры. Более того, интерпретатор кэширует их, поэтому на создание новых уходит меньше времени по сравнению со списками (покуда размер кортежа меньше 20).

Изменение размера кортежа возможно только путем создания нового путем конкатенации:

In [4]:
t1 = (1,2,3,4)
t2 = (5,6,7,8)
print id(t1)
print id(t2)
t1 += t2
print id(t1)
print id(t2)
print t1

140159281643200
140159281640648
140159282421464
140159281640648
(1, 2, 3, 4, 5, 6, 7, 8)


In [14]:
(x**2 for x in range(1, 10))

<generator object <genexpr> at 0x7f99d85fa820>

-- В: Что быстрее - list.append(x) или создание нового кортежа? 

In [10]:
%timeit l = [0,1,2,3,4,5,6,7,8,9]

10000000 loops, best of 3: 148 ns per loop


NameError: name 'l' is not defined

In [6]:
%timeit t = (0,1,2,3,4,5,6,7,8,9)

10000000 loops, best of 3: 20 ns per loop


-- N: list comprehension

In [12]:
type([x**2 for x in xrange(10)])

list

In [10]:
[x**2 for x in xrange(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

## TLDR:
- список использует больше памяти, однако с точки зрения постоянного добавления новых данных - более быстрая структура;
- кортеж эффективный занимает память, быстрее создается, однако он - статический.

# 2.2. Словари и множества (dict, set)

Словари и множества используют схожий механимз для обеспечения быстрой проверки на наличие в них элемента: хэш - таблицу. Если хэш функция - "хорошая", то доступ к элементу осуществляется за константное время. Минимальный размер словаря - 8 элементов, после ресайза он увеличивается в 4 раза до 50 000 элементов , и затем в 2 раза. 

Для того, чтобы иметь возможность хэшировать произвольные классы, у класса должны быть определены методы \_\_hash\_\_ и \_\_cmp\_\_. По умолчанию, \_\_hash\_\_ возвращает адрес в памяти, а \_\_cmp\_\_ сравнивает эти значения.

In [11]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

p1 = Point(1,1)
p2 = Point(1,1)

set([p1, p2])

{<__main__.Point at 0x7f99d859fa10>, <__main__.Point at 0x7f99e21d5450>}

In [12]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y 
    def __hash__(self):
        return hash((self.x, self.y)) 
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
p1 = Point(1,1)
p2 = Point(1,1)

set([p1, p2])

{<__main__.Point at 0x7f99d85f9b10>}

** Методы dict(): **

** Методы set():**

-- N: dict comprehension

In [10]:
{n: n**2 for n in xrange(5)}

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

In [11]:
{n: n**2 for n in xrange(10) if n % 2 != 0}

{1: 1, 3: 9, 5: 25, 7: 49, 9: 81}

## TLDR:
- если ваш код предполагает проверку на наличие в множестве, либо поиск по ключу - используйте множество и словарь.
- данные в словаре и множестве неупорядочены.

# 2.3. Collections

Данный модуль содержит "альтернативы" стандартным элементам коллекции, рассмотрим их по порядку:

**namedtuple**:

In [12]:
from collections import namedtuple
Point = namedtuple('Point', ['x', 'y'], verbose=True)

class Point(tuple):
    'Point(x, y)'

    __slots__ = ()

    _fields = ('x', 'y')

    def __new__(_cls, x, y):
        'Create new instance of Point(x, y)'
        return _tuple.__new__(_cls, (x, y))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Point object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 2:
            raise TypeError('Expected 2 arguments, got %d' % len(result))
        return result

    def __repr__(self):
        'Return a nicely formatted representation string'
        return 'Point(x=%r, y=%r)' % self

    def _asdict(self):
        'Return a new OrderedDict which maps field names to their values'
        return OrderedDict(zip(self._fields, self))

    def _replace(_self, **kwds):
        'Return a new Point object replacing specified fields with new values'
        result = _self._make(map(kwds.pop, ('x', 'y'), _self))
        if kwds:
            raise ValueError('

** defaultdict **:

In [13]:
from collections import defaultdict
def_dict = defaultdict(list)

In [14]:
def_dict['a'].append(5)
def_dict['b']

[]

In [15]:
for k, v in def_dict.iteritems():
    print k, v

a [5]
b []


**ordereddict**:

In [16]:
from collections import OrderedDict

In [17]:
od = OrderedDict()
od['b'] = 5
od['c'] = 3

for k,v in od.iteritems():
    print k,v

b 5
c 3


**deque**:

In [18]:
from collections import deque
d = deque('ghi')
d.append('j')
d.appendleft('f')
print d.pop()
print d.popleft()

j
f


**counter**:

In [18]:
from collections import Counter
c = Counter()
print id(c)
c = Counter([1,23,1,23,4,566,7744,21,1,2,1])
print id(c)

140298736845176
140298736845768


In [16]:
c

Counter({1: 4, 2: 1, 4: 1, 21: 1, 23: 2, 566: 1, 7744: 1})

In [17]:
c.most_common()

[(1, 4), (23, 2), (7744, 1), (2, 1), (4, 1), (21, 1), (566, 1)]

# 2.4. Итераторы и генераторы

In [22]:
def do_work(x):
    print x

objects = [1,2,3,4,5]

for i in objects: 
    do_work(i)
    
print 
# то же самое

object_iterator = iter(objects) 
while True:
    try:
        i = object_iterator.next()
        do_work(i)
    except StopIteration:
        break

1
2
3
4
5

1
2
3
4
5


Это возможно благодаря т.н. протоколу итерации (iteration protocol). 

In [23]:
x = iter([1, 2, 3])

In [24]:
x.next()

1

In [25]:
class Count_Between(object):
    def __init__(self, low, high):
        self.low = low
        self.high = high
    def __iter__(self):
        counter = self.low
        while self.high >= counter:
            yield counter
            counter += 1

In [26]:
gobj = Count_Between(5, 10)
for num in gobj:
    print(num)

5
6
7
8
9
10


Как вы уже догадались - это возможно благодаря тому, что у объекта реализован метод \_\_iter\_\_. Генератор облегчает создание итераторов. Генератор возвращает результат "порционно" вместо единственного значение. Генератор объявляется как функция, за исключением того, что использует ключевое слово yield вместо return.

In [25]:
def fibonacci():
    i,j=0,1
    while True:
        yield j
        i,j=j,i+j

In [26]:
from itertools import islice
first_5000 = islice(fibonacci(), 0, 5000)

In [27]:
sum(first_5000)

1015527125487728271973716941667558936756065641738270254518642166382348739585570036148670662799001691980927814310872979361413168209199509052507176284450804112343785698623090615014712143123968126951333374199952106383340704572994599080314353436374887459555984981306202677747047063839632335530206432495010511259085556507532520713290125222114160175309162010588500278254127298052343374867913372505695428504864515646591379109165056157033950874536186106150482303604978936702039498126711424296314600743272155756297215283842943009470400137487102595682275817428741714454705554787529433671658876889220852274956174911790771356713661745667955431129460342847653779324049369668962683249216479089866850735160553205932622200667658562756343414170187173881137412028498345881150895852017447502470958861220538316680468334243551031072726267934106753500099074964060723274095084204735840671108216398525636637271635861216857478551669726239508419251684940028284780528548621090905811380338386977189978164852554867605990468717644

По аналогии с list comprehension, можно сделать т.н. generator expression. Значение будет посчитано "на лету", исчерпав итератор мы не сможем повторить вычисления снова, тем не менее значительно выиграем в расходах по памяти.  

In [21]:
is_odd = lambda x: x % 2

In [28]:
sum(1 for x in first_5000 if is_odd(x))

0

In [29]:
sum(x*x for x in range(1,10))

285

Хороший разбор пример использования итераторов и генераторов можно найти тут: http://www.dabeaz.com/generators-uk/ .

## TLDR:
- везде, где это возможно - используйте генераторы;
- избегайте избыточного создания промежуточных объектов - во многих местах достаточно ограничится итератором, а не создавать новый объект с определенными свойствами.

# 2.5. itertools

Данный модуль содержит очень полезные итераторы, значительно облегчающие работу с данными (https://docs.python.org/2/library/itertools.html).

** islice**: позволяет работать с "бесконечными" генераторами.

In [33]:
from itertools import islice

# seq, [start,] stop [, step]

list(islice('ABCDEFG', 2, None))

['C', 'D', 'E', 'F', 'G']

** takewhile **: добавяляет условие для остановки генератора

In [34]:
from itertools import takewhile

# pred, seq

list(takewhile(lambda x: x<5, [1,4,6,4,1]))

[1, 4]

**cycle**: сделать генератор бесконечным, все время его повторяя

In [30]:
from itertools import cycle

inf = cycle('ABCD')    

In [38]:
inf.next()

'D'

**chain**: сделать композицию итераторов

In [39]:
from itertools import chain

list(chain('ABC', 'DEF'))

['A', 'B', 'C', 'D', 'E', 'F']

**groupby**: сгрупировать итераторы

In [40]:
from itertools import groupby

# iterable[, keyfunc]

test_inp = (('a 213123', 1), ('a 124124', 2), ('a 12312', 10), ('b 12421', 3))
for el in groupby(test_inp, key = lambda x: x[0].split(' ')[0]):
    print el

('a', <itertools._grouper object at 0x7f99d84a61d0>)
('b', <itertools._grouper object at 0x7f99d84a6290>)


** imap: **

In [39]:
from itertools import imap

imap(pow, (2,3,10), (5,2,3))

** ifilter **:

In [40]:
from itertools import ifilter

for el in ifilter(lambda x: x%2, range(10)):
    print el

1
3
5
7
9


# 2.6. functools

**map()**:
создает новый list() путем применения функции f()  к исходному iterable массиву:

In [41]:
map(lambda x: x**2, (1,2,3))

[1, 4, 9]

**filter()**: создает новый массив, который состоит из объектов, удовлетворяющих предикату.

In [42]:
filter(lambda x: x%2 == 0, xrange(20))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

**sum()**: 

In [42]:
sum(xrange(10))

45

**reduce()**:

In [44]:
reduce(lambda x,y: (x+y), [2,4,6])

12

**partial():**

In [45]:
from functools import partial

basetwo = partial(int, base=2)

In [46]:
basetwo('10010')

18

# Дополнительный материал 3. Поиск в пространстве имен.

Как мы уже знаем, поиск объектов ведется по правилу LEGB, с использованием словарей locals() и globals().

In [47]:
import math
from math import sin

def test1(x):
    return math.sin(x)

def test2(x):
    return sin(x)

def test3(x, sin=math.sin):
    return sin(x)

In [48]:
%timeit test1(123456)

The slowest run took 40.40 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 224 ns per loop


In [49]:
%timeit test2(123456)

The slowest run took 18.25 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 170 ns per loop


In [50]:
%timeit test3(123456)

The slowest run took 12.26 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 175 ns per loop


Питон всегда пытается максимально ускорить доступ к локальным данным, поэтому перенос функции в locals() может значительно ускорить время ее выполнения.